# Exploratory Data Analysis - Improved

Comprehensive EDA for cat breed classification dataset with:
- Dataset statistics and distribution analysis
- Visual exploration of images
- Class imbalance detection
- Sample image visualization
- Recommendations for data preprocessing

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import random
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Import config
sys.path.append('..')
import config

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

%matplotlib inline

## 1. Load and Explore Dataset Statistics

In [ ]:
# Check if dataset stats CSV exists
if config.DATASET_STATS_CSV.exists():
    df_stats = pd.read_csv(config.DATASET_STATS_CSV)
    print("✓ Loaded dataset statistics")
    print(f"\nDataset shape: {df_stats.shape}")
    df_stats.head(10)
else:
    print(f"⚠ Dataset stats not found at {config.DATASET_STATS_CSV}")
    df_stats = None

In [ ]:
if df_stats is not None:
    print("="*80)
    print("DATASET SUMMARY STATISTICS")
    print("="*80)
    print(df_stats.describe())

## 2. Analyze Raw Data Distribution

In [ ]:
def scan_directory(directory):
    """Scan directory and count images per breed"""
    breed_counts = {}
    total_images = 0
    
    if not directory.exists():
        print(f"⚠ Directory not found: {directory}")
        return breed_counts, total_images
    
    for breed_dir in directory.iterdir():
        if breed_dir.is_dir():
            breed_name = breed_dir.name
            image_files = [
                f for f in breed_dir.iterdir()
                if f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
            ]
            count = len(image_files)
            breed_counts[breed_name] = count
            total_images += count
    
    return breed_counts, total_images

# Scan raw data
print("Scanning raw data directory...")
raw_counts, raw_total = scan_directory(config.RAW_DATA_DIR)
print(f"✓ Found {len(raw_counts)} breeds, {raw_total} images")

# Scan cleaned data
print("\nScanning cleaned data directory...")
cleaned_counts, cleaned_total = scan_directory(config.CLEANED_DATA_DIR)
print(f"✓ Found {len(cleaned_counts)} breeds, {cleaned_total} images")

## 3. Visualize Class Distribution

In [ ]:
if cleaned_counts:
    # Create dataframe
    df_dist = pd.DataFrame({
        'breed': list(cleaned_counts.keys()),
        'count': list(cleaned_counts.values())
    }).sort_values('count', ascending=False)
    
    # Plot distribution
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    
    # 1. Bar plot (sorted)
    axes[0, 0].bar(range(len(df_dist)), df_dist['count'], color='steelblue')
    axes[0, 0].set_xlabel('Breed (sorted by count)', fontsize=12)
    axes[0, 0].set_ylabel('Number of Images', fontsize=12)
    axes[0, 0].set_title('Image Count per Breed (Sorted)', fontsize=14, fontweight='bold')
    axes[0, 0].axhline(y=df_dist['count'].mean(), color='red', linestyle='--', label=f'Mean: {df_dist["count"].mean():.1f}')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Histogram
    axes[0, 1].hist(df_dist['count'], bins=20, color='coral', edgecolor='black')
    axes[0, 1].set_xlabel('Number of Images', fontsize=12)
    axes[0, 1].set_ylabel('Frequency', fontsize=12)
    axes[0, 1].set_title('Distribution of Image Counts', fontsize=14, fontweight='bold')
    axes[0, 1].axvline(x=df_dist['count'].mean(), color='red', linestyle='--', label=f'Mean: {df_dist["count"].mean():.1f}')
    axes[0, 1].axvline(x=df_dist['count'].median(), color='green', linestyle='--', label=f'Median: {df_dist["count"].median():.1f}')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Top 20 breeds
    top_20 = df_dist.head(20)
    axes[1, 0].barh(range(len(top_20)), top_20['count'], color='lightgreen')
    axes[1, 0].set_yticks(range(len(top_20)))
    axes[1, 0].set_yticklabels([b.replace('_', ' ').title() for b in top_20['breed']])
    axes[1, 0].set_xlabel('Number of Images', fontsize=12)
    axes[1, 0].set_title('Top 20 Breeds by Image Count', fontsize=14, fontweight='bold')
    axes[1, 0].invert_yaxis()
    axes[1, 0].grid(True, alpha=0.3, axis='x')
    
    # 4. Bottom 20 breeds
    bottom_20 = df_dist.tail(20)
    axes[1, 1].barh(range(len(bottom_20)), bottom_20['count'], color='salmon')
    axes[1, 1].set_yticks(range(len(bottom_20)))
    axes[1, 1].set_yticklabels([b.replace('_', ' ').title() for b in bottom_20['breed']])
    axes[1, 1].set_xlabel('Number of Images', fontsize=12)
    axes[1, 1].set_title('Bottom 20 Breeds by Image Count', fontsize=14, fontweight='bold')
    axes[1, 1].invert_yaxis()
    axes[1, 1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig(config.PLOTS_DIR / 'eda_class_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n" + "="*80)
    print("CLASS DISTRIBUTION STATISTICS")
    print("="*80)
    print(f"Total breeds: {len(df_dist)}")
    print(f"Total images: {df_dist['count'].sum()}")
    print(f"Mean images per breed: {df_dist['count'].mean():.2f}")
    print(f"Median images per breed: {df_dist['count'].median():.2f}")
    print(f"Std deviation: {df_dist['count'].std():.2f}")
    print(f"Min images: {df_dist['count'].min()} ({df_dist.iloc[-1]['breed']})")
    print(f"Max images: {df_dist['count'].max()} ({df_dist.iloc[0]['breed']})")
    print(f"\nImbalance ratio: {df_dist['count'].max() / df_dist['count'].min():.2f}x")

## 4. Check for Class Imbalance

In [ ]:
if cleaned_counts:
    # Calculate imbalance metrics
    mean_count = df_dist['count'].mean()
    std_count = df_dist['count'].std()
    
    # Classes with significantly fewer samples
    underrepresented = df_dist[df_dist['count'] < mean_count - std_count]
    
    # Classes with significantly more samples
    overrepresented = df_dist[df_dist['count'] > mean_count + std_count]
    
    print("\n" + "="*80)
    print("CLASS IMBALANCE ANALYSIS")
    print("="*80)
    
    print(f"\n📉 Underrepresented breeds ({len(underrepresented)} breeds):")
    print(f"   (< {mean_count - std_count:.0f} images)\n")
    for idx, row in underrepresented.iterrows():
        print(f"   {row['breed']:30s} {row['count']:3d} images")
    
    print(f"\n📈 Overrepresented breeds ({len(overrepresented)} breeds):")
    print(f"   (> {mean_count + std_count:.0f} images)\n")
    for idx, row in overrepresented.iterrows():
        print(f"   {row['breed']:30s} {row['count']:3d} images")
    
    print(f"\n⚖️  Balanced breeds: {len(df_dist) - len(underrepresented) - len(overrepresented)} breeds")
    
    # Recommendations
    print("\n" + "="*80)
    print("RECOMMENDATIONS")
    print("="*80)
    if len(underrepresented) > 0:
        print("⚠  Consider data augmentation for underrepresented classes")
    if len(overrepresented) > 0:
        print("⚠  Consider downsampling or class weights for overrepresented classes")
    if df_dist['count'].std() / df_dist['count'].mean() > 0.3:
        print("⚠  High class imbalance detected - use stratified sampling")
    else:
        print("✓ Dataset is relatively balanced")

## 5. Sample Image Visualization

In [ ]:
def visualize_random_samples(data_dir, num_breeds=6, images_per_breed=3):
    """
    Visualize random sample images from different breeds
    """
    if not data_dir.exists():
        print(f"⚠ Directory not found: {data_dir}")
        return
    
    # Get random breeds
    breed_dirs = [d for d in data_dir.iterdir() if d.is_dir()]
    if len(breed_dirs) == 0:
        print("No breed directories found")
        return
    
    selected_breeds = random.sample(breed_dirs, min(num_breeds, len(breed_dirs)))
    
    fig, axes = plt.subplots(num_breeds, images_per_breed, figsize=(15, num_breeds * 4))
    if num_breeds == 1:
        axes = axes.reshape(1, -1)
    
    for i, breed_dir in enumerate(selected_breeds):
        breed_name = breed_dir.name
        
        # Get random images
        image_files = [
            f for f in breed_dir.iterdir()
            if f.suffix.lower() in ['.jpg', '.jpeg', '.png']
        ]
        
        if len(image_files) == 0:
            continue
        
        selected_images = random.sample(image_files, min(images_per_breed, len(image_files)))
        
        for j, img_path in enumerate(selected_images):
            try:
                img = Image.open(img_path)
                axes[i, j].imshow(img)
                axes[i, j].axis('off')
                
                if j == 0:
                    axes[i, j].set_title(
                        f"{breed_name.replace('_', ' ').title()}\n{img.size[0]}×{img.size[1]}px",
                        fontsize=12,
                        fontweight='bold',
                        loc='left'
                    )
                else:
                    axes[i, j].set_title(f"{img.size[0]}×{img.size[1]}px", fontsize=10)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                axes[i, j].axis('off')
    
    plt.suptitle('Random Sample Images from Different Breeds', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(config.PLOTS_DIR / 'eda_sample_images.png', dpi=300, bbox_inches='tight')
    plt.show()

# Visualize samples
print("Visualizing random sample images...")
visualize_random_samples(config.CLEANED_DATA_DIR, num_breeds=6, images_per_breed=3)

## 6. Image Size Analysis

In [ ]:
def analyze_image_sizes(data_dir, num_samples=500):
    """
    Analyze image dimensions across the dataset
    """
    if not data_dir.exists():
        print(f"⚠ Directory not found: {data_dir}")
        return
    
    widths = []
    heights = []
    aspect_ratios = []
    
    # Collect all image paths
    all_images = []
    for breed_dir in data_dir.iterdir():
        if breed_dir.is_dir():
            all_images.extend([
                f for f in breed_dir.iterdir()
                if f.suffix.lower() in ['.jpg', '.jpeg', '.png']
            ])
    
    # Sample images
    sampled_images = random.sample(all_images, min(num_samples, len(all_images)))
    
    print(f"Analyzing {len(sampled_images)} images...")
    
    for img_path in sampled_images:
        try:
            with Image.open(img_path) as img:
                w, h = img.size
                widths.append(w)
                heights.append(h)
                aspect_ratios.append(w / h)
        except:
            continue
    
    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Width distribution
    axes[0, 0].hist(widths, bins=30, color='skyblue', edgecolor='black')
    axes[0, 0].set_xlabel('Width (pixels)', fontsize=12)
    axes[0, 0].set_ylabel('Frequency', fontsize=12)
    axes[0, 0].set_title('Image Width Distribution', fontsize=14, fontweight='bold')
    axes[0, 0].axvline(x=np.mean(widths), color='red', linestyle='--', label=f'Mean: {np.mean(widths):.0f}')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Height distribution
    axes[0, 1].hist(heights, bins=30, color='lightcoral', edgecolor='black')
    axes[0, 1].set_xlabel('Height (pixels)', fontsize=12)
    axes[0, 1].set_ylabel('Frequency', fontsize=12)
    axes[0, 1].set_title('Image Height Distribution', fontsize=14, fontweight='bold')
    axes[0, 1].axvline(x=np.mean(heights), color='red', linestyle='--', label=f'Mean: {np.mean(heights):.0f}')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Scatter plot
    axes[1, 0].scatter(widths, heights, alpha=0.5, s=10)
    axes[1, 0].set_xlabel('Width (pixels)', fontsize=12)
    axes[1, 0].set_ylabel('Height (pixels)', fontsize=12)
    axes[1, 0].set_title('Width vs Height', fontsize=14, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Aspect ratio
    axes[1, 1].hist(aspect_ratios, bins=30, color='lightgreen', edgecolor='black')
    axes[1, 1].set_xlabel('Aspect Ratio (W/H)', fontsize=12)
    axes[1, 1].set_ylabel('Frequency', fontsize=12)
    axes[1, 1].set_title('Aspect Ratio Distribution', fontsize=14, fontweight='bold')
    axes[1, 1].axvline(x=np.mean(aspect_ratios), color='red', linestyle='--', label=f'Mean: {np.mean(aspect_ratios):.2f}')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(config.PLOTS_DIR / 'eda_image_sizes.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n" + "="*80)
    print("IMAGE SIZE STATISTICS")
    print("="*80)
    print(f"Width:  Mean={np.mean(widths):.1f}, Median={np.median(widths):.1f}, Std={np.std(widths):.1f}")
    print(f"Height: Mean={np.mean(heights):.1f}, Median={np.median(heights):.1f}, Std={np.std(heights):.1f}")
    print(f"Aspect Ratio: Mean={np.mean(aspect_ratios):.2f}, Median={np.median(aspect_ratios):.2f}")
    print(f"\nMin size: {min(widths)}×{min(heights)}")
    print(f"Max size: {max(widths)}×{max(heights)}")

# Analyze sizes
analyze_image_sizes(config.CLEANED_DATA_DIR, num_samples=500)

## 7. Summary and Recommendations

In [ ]:
print("="*80)
print("EDA SUMMARY AND RECOMMENDATIONS")
print("="*80)

print("\n📊 DATASET OVERVIEW:")
print(f"   • Total breeds: {len(cleaned_counts)}")
print(f"   • Total images: {cleaned_total}")
print(f"   • Average images per breed: {np.mean(list(cleaned_counts.values())):.1f}")

print("\n✅ STRENGTHS:")
print("   • Large number of breeds (63) for fine-grained classification")
print("   • YOLO-based cleaning ensures image quality")
print("   • No corrupt files")

print("\n⚠️  CHALLENGES:")
if len(underrepresented) > 0:
    print(f"   • Class imbalance: {len(underrepresented)} underrepresented breeds")
print("   • Fine-grained classification (similar breeds may be hard to distinguish)")
print("   • Variable image sizes require resizing")

print("\n🔧 RECOMMENDATIONS:")
print("   1. Use stratified train/val/test split")
print("   2. Apply data augmentation (especially for underrepresented classes)")
print("   3. Resize all images to 224×224 for ResNet50V2")
print("   4. Consider class weights during training to handle imbalance")
print("   5. Use transfer learning from ImageNet pretrained models")
print("   6. Monitor both accuracy and top-5 accuracy metrics")
print("   7. Use confusion matrix to identify commonly confused breeds")

print("\n" + "="*80)
print("✓ EDA Complete! Proceed to data preprocessing and model training.")
print("="*80)